<a href="https://colab.research.google.com/github/DAOTHITHUNGA/CS114.K21.KHTN/blob/master/Sarcasm_Detec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###BÁO CÁO BÀI TẬP LỚN 
Đào Thị Thu Nga - 18521135


Mô tả bài toán : 
* data : Lấy headlines ở 2 trang chính là:
+ huffpost : chứa các tin không châm biếm 
+ TheOnion : chứa các tin châm biếm (is sarcastic)
* Gồm 3 thuộc tính: 
  + article_link : đường link dẫn tới bài báo 
  + is_sarcastic : xét có phải là châm biếm hay không với 2 option ( 1 nếu là châm biếm , 0 nếu là không châm biếm )
  + headlines : tên tiêu đề bài báo. 

### Từ tiêu đề của một bài báo, dự đoán bài báo đó có mang nội dung cà khịa hay không.

#### Cách thu thập dữ diệu:
-  Trước đây về Phát hiện Sarcasm chủ yếu sử dụng các bộ dữ liệu Twitter được thu thập bằng cách dựa trên hashtag nhưng các bộ dữ liệu đó rất nhiễu về LABEL và ngôn ngữ. Hơn nữa, nhiều tweet là trả lời các tweet khác và phát hiện sự mỉa mai trong những điều này đòi hỏi phải có sẵn các tweet theo ngữ cảnh. Để khắc phục những hạn chế liên quan đến  nhiễu trong bộ dữ liệu Twitter.

> Indented block


- Dữ liệu mới được thu thập để giảm độ nhiễu hơn so với dữ liệu thu thập từ Tweet từ trước. Dữ liệu được thu thập từ 2 trang tin tức chính:
    - Tập mang tính mỉa mai : Được thu thập từ trang tin tức TheOnion, từ 2 chủ đề chính là News in Brief và News in Photos
    - Tập không mang tính mỉa mai : Được thu thập từ trang tin tức HuffPost.
- So với dữ liệu cũ, dữ liệu mới có một số cải thiện:
    - Được viết bởi những nhà báo chuyên nghiệp, không mắc các lỗi ngữ pháp, có quy chuẩn. Giảm thiểu việc phải tiền xử lý dữ liệu.



###Cách lấy dữ liệu từ 2 trang huffpost và TheOnion 
Lấy file json của trang huffpost 
  - 1 lần  chỉ lấy được 500 line 
- Thực hiện các thao tác lấy file json của huffpost 
  - Lần 1 : Lấy 500 headline ở https://www.huffpost.com/api/department/news/cards?page=1&limit=500
  - Lần 2: Lấy 143 headline ở https://www.huffpost.com/api/department/news/cards?page=2&limit=500
Lấy dữ liệu từ trang TheOnion 
+ News in Brief và News in Photos

###CRAWL 2000 DỮ LIỆU 

In [1]:
#import thư viện cần thiết 
import json
import csv
import pandas as pd


Xử lí file json của huffpost

In [3]:
#mở file chứa 158 headlines
with open('/content/huffpost.json') as f:
    part1 = json.load(f)
#mở file chứa 500 headlines
with open('/content/huffpost2.json') as f:
  part2 = json.load(f)
#tạo file data tên huffpost_headlines.
csv_file = open('huffpost_headlines.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['article_link','headlines', 'is_sarcastic'])
for article in part1["cards"]:
  headline = article["headlines"][0]["text"]
  article_link = article["headlines"][0]["url"]
  csv_writer.writerow([article_link,headline,0])
    
for article in part2["cards"]:
  headline = article["headlines"][0]["text"]
  article_link = article["headlines"][0]["url"]
  csv_writer.writerow([article_link,headline,0])
  #csv_file.close()

In [4]:
#đọc file huffpost_headlines.csv
huffpost_df = pd.read_csv("huffpost_headlines.csv")
huffpost_df.head()

,article_link,headlines,is_sarcastic
0,https://www.huffpost.com/entry/prince-william-...,Prince William Steps Out To Visit COVID-19 Vac...,0
1,https://www.huffpost.com/entry/supreme-court-d...,Supreme Court Bolsters Trump's Power Over Rapi...,0
2,https://www.huffpost.com/entry/george-floyd-di...,George Floyd Died A Month Ago. There Have Been...,0
3,https://www.huffpost.com/entry/carly-fiorina-2...,Former GOP Candidate Carly Fiorina Says She Wo...,0
4,https://www.huffpost.com/entry/colorado-elijah...,Colorado Governor Will Examine Death Of Elijah...,0


#Chuyển đổi từ file csv sang json

In [5]:
import csv
import json

csvfile = open('/content/huffpost_headlines.csv', 'r')
jsonfile = open('file.json', 'w')

fieldnames = ("article_link","headlines", "is_sarcastic")
reader = csv.DictReader( csvfile, fieldnames)
for row in reader:
    json.dump(row, jsonfile)
    jsonfile.write('\n')

#CRAWL dữ liệu từ TheOnion

In [7]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import csv

brief_ids = [1591290240502,1590689280047,1589990700904,1589468640865,1588797540738,1588268460534,
             1587475140185,1586891580286,1586290320129,1585667400574,
             1584992760722,1584481080367,1583943300489,1583433840399,1583174280261,
             1582734480159,1582230180154,1581691680914,1581347940240,
              1580846940042,1580238780534,1579722060702,1579199040121,1578667500624,
              1576785600904,1576524120524,1575914280494,1575386760500,1574446260671,
              1573853280300,1573574280081,1573065060211,1572465120742,1572010200227,
              1571684700481,1571189760863,1570808100042,1570464000244,1570017600467,1569521460355,1569256200911
              ,1568653560896,1568141220732,1567616220385,1567090800228,1566577200032,
              1566249480367,1565716500732,1565282640227,1564762560287,1564500720500,1563984120219,1563541200063
             ,1563219960624,1562682300628]

photo_ids = [1590159240893,1588856760531,1587659280048,1586191620240,1584377100135,1582815480621,1582135980235,1580919900163,1579630380938
            ,1576675800080,1574713020966,1573241220619,1571856060048,1570206360590,1569267060072,1567600200783,1565965800980,
            1564618440716,1563282120008,1561045680206,1559133960526,1557427680731,1556198100070,
            1554825000123,1552924740401,1551208620373,1550168460985,1549035480816,
            1547746620251,1544817960808,1543529340295,1541717040221]

csv_file = open('TheOnion.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headlines', 'is_sarcastic'])

24

In [8]:
def Onion_crawler(tag, ids, num_articles, label=1):
  count = 0
  for id in ids:
    source = requests.get("https://www.theonion.com/c/{}?startTime={}".format(tag,id)).text
    soup = BeautifulSoup(source, 'lxml')
    for article in soup.find_all('article'):
      headline = article.h2.text
      csv_writer.writerow([headline,1])
      count += 1
      if count == num_articles:
        print(f"Crawled {count} headlines from {tag} tag")
        return

In [9]:
Onion_crawler("news-in-brief", brief_ids, num_articles=750, label=1)
Onion_crawler("news-in-photos", photo_ids, num_articles=610, label=1)
csv_file.close()

Crawled 750 headlines from news-in-brief tag
Crawled 610 headlines from news-in-photos tag


In [10]:
#đọc file TheOnion
onion_df = pd.read_csv("TheOnion.csv")
onion_df.head(10)

,headlines,is_sarcastic
0,Drew Brees Admits He Doesn’t Trust Black Peopl...,1
1,Mark Zuckerberg Announces Virtual Roundtable W...,1
2,‘This Face Will Be The Last Thing You See Befo...,1
3,Primaried Steve King Glad He At Least Won’t Ha...,1
4,Police Defend Use Of Non-Lethal Rubber Tires O...,1
5,Democratic Leaders Announce That They’ve Learn...,1
6,"‘She Needs A Bible Now, Fuckwad—Yes, It’s For ...",1
7,Auntie Anne’s Breaks From Pack By Calling For ...,1
8,Trump Throws Garbage Can Through McDonald’s Wi...,1
9,Police Didn’t Spend Millions On Awesome Tank J...,1


In [11]:
#Chuyển đổi từ file csv sang file json
import csv
import json

csvfile = open('/content/TheOnion.csv', 'r')
jsonfile = open('Mydata_TheOnion.json', 'w')

fieldnames = ("headlines", "is_sarcastic")
reader = csv.DictReader( csvfile, fieldnames)
for row in reader:
    json.dump(row, jsonfile)
    jsonfile.write('\n')

Em trộn 2 file json của 2 trang Huffpost và TheOnion vào file 2000_test_headlines.json

### kết nối gg drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### import thư viện cần thiết 

In [ ]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#đọc file json của kaggle (file train)

In [ ]:
# Loading train data from json file
data_train = pd.read_json("/content/drive/My Drive/hoctap/sarcasm/Sarcasm_Headlines_Dataset.json", lines = True)

#kiểm tra 

In [ ]:
print(data_train.isnull().any(axis = 0))
# Output :-
# ----------------------
# article_link    False
# headline        False
# is_sarcastic    False
# dtype: bool
# ----------------------

article_link    False
headline        False
is_sarcastic    False
dtype: bool


In [ ]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
#LOẠI BỎ CÁC KÍ TỰ ĐẶC BIỆT KHÔNG CẦN THIẾT TRÊN TẬP TEST 
data_train['headline'] = data_train['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
# getting features and labels
features_train = data_train['headline']
labels_train = data_train['is_sarcastic']
# Stemming our data
ps = PorterStemmer()
features_train = features_train.apply(lambda x: x.split())
features_train = features_train.apply(lambda x : " ".join([ps.stem(word) for word in x]))

In [ ]:
# vectorizing the data with maximum of 5000 features
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features_train = list(features_train)
features_train = tv.fit_transform(features_train).toarray()

In [ ]:
# Loading test data from THUNGADETHUONG WITH 2000 headlines
testdata = pd.read_json("/content/drive/My Drive/hoctap/sarcasm/2000_test_headlines.json", lines = True)

In [ ]:
print(testdata.isnull().any(axis = 0))
#KIỂM TRA TẬP TEST 

article_link     True
headlines       False
is_sarcastic    False
dtype: bool


In [ ]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
#LOẠI BỎ CÁC KÍ TỰ ĐẶC BIỆT KHÔNG CẦN THIẾT TRÊN TẬP TEST 
testdata['headlines'] = testdata['headlines'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
# getting features and labels
#LẤY FEATURES VÀ LABEL CHO TẬP TEST 
features_test = testdata['headlines']
labels_test = testdata['is_sarcastic']
# Stemming our data
ps = PorterStemmer()
features_test = features_test.apply(lambda x: x.split())
features_test = features_test.apply(lambda x : " ".join([ps.stem(word) for word in x]))

In [ ]:
# feature extraction 
# Khai báo TfidfVectorizer để biểu diễn dữ liệu train dưới dạng vector và tạo một từ điển từ dữ liệu train.
# Sử dụng lại từ điển của dữ liệu trên để biểu diễn dữ liệu test dưới dạng vector
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features_test = list(features_test)
features_test = tv.fit_transform(features_test).toarray()

In [ ]:
# model 1:-
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))   

0.9075966902542214
0.5158730158730159


In [ ]:
# model 2:-
# Using Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('Using Gaussuan Naive Bayes')
print(gnb.score(features_train, labels_train)) 
print(gnb.score(features_test, labels_test))   
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('Logistic Regression')
print(lr.score(features_train, labels_train))  
print(lr.score(features_test, labels_test))   
# model 4:-
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('Random Forest Classifier')
print(rfc.score(features_train, labels_train)) 
print(rfc.score(features_test, labels_test))   

Using Gaussuan Naive Bayes
0.7844172376352541
0.5104166666666666
Logistic Regression
0.880265079186791
0.5054563492063492
Random Forest Classifier
0.9885431876895429
0.5500992063492064


### CHIA TẬP TRAIN VÀ TEST TRÊN TẬP DATA KAGGLE

In [ ]:
features=features_train
labels=labels_train
#getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

In [ ]:
# model 1:-
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print(' Using linear support vector classifier')
print(lsvc.score(features_train, labels_train)) # 90.93
print(lsvc.score(features_test, labels_test))   # 83.75
# model 2:-
# Using Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('Using Gaussuan Naive Bayes')
print(gnb.score(features_train, labels_train))  # 78.86
print(gnb.score(features_test, labels_test))    # 73.80
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('Logistic Regression')
print(lr.score(features_train, labels_train))   # 88.16
print(lr.score(features_test, labels_test))     # 83.08
# model 4:-
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('Random Forest Classifier')
print(rfc.score(features_train, labels_train))  # 98.82
print(rfc.score(features_test, labels_test))    # 79.71

 Using linear support vector classifier
0.9093524612777362
0.8375748502994012
Using Gaussuan Naive Bayes
0.7886335868836952
0.7380239520958084
Logistic Regression
0.8816458440074094
0.8308383233532934
Random Forest Classifier
0.9882946439128207
0.7971556886227545


### Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới. Nêu nhận xét của nhóm về bài toán này.

- Từ kết quả trên thì performance của model trên dataset đã cho có kết quả cao hơn 2000 headline mới.
- Sử dụng bag of word không giúp hiểu nội dung headlines. Vì vậy cần tìm hiểu và test 1 số thuật toán khác như trong 1 bài báo sử dụng WORD2VEC MODEL và GloVe Embeddings để cải thiện performance hơn.

###Mô tả cách dùng model đã train để viết một đoạn chương trình ngắn, thực hiện sacarsm detection cho một headline bất kỳ được nhập vào.

In [ ]:
import re
def pre_process(headline): #preprecessing data
	headline = headline.lower()                                 
	headline = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", headline)
	headline = re.sub("(\\d|\\WZhe Yang)+"," ", headline)                   
	return headline
data_train['headline'] = data_train['headline'].apply(lambda x:pre_process(x))
data_train['headline']

0        former versace store clerk sues over secret  b...
1        the  roseanne  revival catches up to our thorn...
2        mom starting to fear son s web series closest ...
3        boehner just wants wife to listen  not come up...
4        j k  rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free fall
26705                              america s best    hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie     
Name: headline, Length: 26709, dtype: object

In [ ]:
headline = input("Enter headline: ")
for cls in [LinearSVC,GaussianNB,LogisticRegression,RandomForestClassifier]:
    print(cls.__class__.__name__, cls.predict(TfidfVectorizer.transform([pre_process(headline)])))

Enter headline: a a 


TypeError: ignored